In [17]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier

In [13]:
data = pd.read_csv('../data/raw/data.csv',  encoding= 'unicode_escape')
y = data["Unusual"]                      #defining the labels
X = data.drop(["Unusual"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
# Load the pipeline from disk using joblib
loaded_pipeline = joblib.load('preprocessing_pipeline.joblib')
X_processed_data = loaded_pipeline.transform(X_train)

In [18]:
# Define the machine learning model
model = RandomForestClassifier(n_estimators=100, max_depth=5)

In [21]:
# Fit the machine learning model to the pre-processed data
model.fit(X_processed_data, y_train)

RandomForestClassifier(max_depth=5)